In [6]:
embeding_matrix = load_pickle_file("../new_disk/embeding_matrix")
word2index = load_pickle_file("../new_disk/word2index")
index_sequences = load_pickle_file("../new_disk/index_sequences.pkl")
labels = load_pickle_file("../Modeling/labels")
SEQUENCE_LENGTH = 434
VOCAB_SIZE = len(embeding_matrix)
EMB_DIMENSION = 300
VALIDATION_SPLIT = 0.1
PAD_SEQ_VALUE = 3000440

In [7]:
import numpy as np
same_lenght_seq = [seq[:SEQUENCE_LENGTH].reshape(1,SEQUENCE_LENGTH) if len(seq) >= SEQUENCE_LENGTH 
                   else np.concatenate((seq,np.array([PAD_SEQ_VALUE] * (SEQUENCE_LENGTH - len(seq)))),axis = 0).reshape(1,SEQUENCE_LENGTH) 
                   for seq in index_sequences]
same_lenght_seq = np.concatenate(same_lenght_seq,axis=0)
labels = np.array(labels)

Dataset split

In [8]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(same_lenght_seq, labels, test_size=0.2, random_state=42,shuffle = True)

In [9]:
data_size = len(x_train)
perm = np.random.permutation(data_size)
idx_train = perm[:int(data_size*(1-VALIDATION_SPLIT))]
idx_val = perm[int(data_size*(1-VALIDATION_SPLIT)):]

data_train = x_train[idx_train]
labels_train = y_train[idx_train]

data_val = x_train[idx_val]
labels_val = y_train[idx_val]

In [10]:
data_train.shape

(7380, 434)

<h1>LSTM</h1>

In [10]:
import tensorflow as tf
from tensorflow import keras as tk
from tensorflow.keras import layers as tfkl
from sklearn.metrics import mean_squared_error,mean_absolute_error

from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from tensorflow.keras.models import Model
# from tensorflow.keras.layers.normalization import BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [11]:
model_checkpoint = ModelCheckpoint("./", save_best_only=True, save_weights_only=True)


In [22]:
LSTM_HIDDEN_DIM_SIZE = 126
rate_drop_lstm = 0.15
rate_drop_lstm = 0.15
lstm_model = tk.Sequential()
embedding_layer = Embedding(VOCAB_SIZE,
        EMB_DIMENSION,
        weights=[embeding_matrix],
        input_length=SEQUENCE_LENGTH,
        trainable=False)
lstm_model.add(embedding_layer)
lstm_model.add(LSTM(LSTM_HIDDEN_DIM_SIZE, activation = 'tanh', dropout=rate_drop_lstm, recurrent_dropout=rate_drop_lstm,return_sequences = True))
lstm_model.add(LSTM(LSTM_HIDDEN_DIM_SIZE//2, activation = 'tanh', dropout=rate_drop_lstm, recurrent_dropout=rate_drop_lstm,return_sequences = True))
lstm_model.add(LSTM(LSTM_HIDDEN_DIM_SIZE//2, activation = 'tanh',dropout=rate_drop_lstm, recurrent_dropout=rate_drop_lstm))
lstm_model.add(Dense(1,activation='sigmoid'))

In [20]:
lstm_model.reset_states()

In [23]:
opt = tk.optimizers.Adam(learning_rate=0.001)

In [ ]:
lstm_model.compile(loss='binary_crossentropy',
        optimizer=opt,
        metrics=['acc'])
lstm_model.fit(data_train,labels_train,epochs = 10,batch_size = 32, validation_data = [data_val,labels_val], callbacks = [model_checkpoint])

Train on 7380 samples, validate on 821 samples
Epoch 1/10
7360/7380 [============================>.] - ETA: 2s - loss: 0.6815 - acc: 0.5484

In [17]:
data_train[0]

array([   3563,       1,    5673,    1283,     185,     186,     635,
            26,      17,     257,     373,     343,     338,    2500,
             7,       1,    9552,       2,      10,     195,    1370,
          1939,      43,     983,      61,     489,   11653,       7,
             1,     353,     257,     373,   18571,    7751,   18298,
          1816,      28,     120,     523,     167,       7,       1,
           881,     287,      71,    3196,    7751,       5,     541,
           411,     168,     667,       7,    2741,   18298,      26,
            17,       4,    9411,     221,      19,       7,       1,
            80,    2811,     242,    2133,     134,    2133,     312,
           122,     250,     164,    1702,     349,     758,     279,
          1132,     316,     186,     635,     281,       5,    1085,
          2261,     166,     403,     449,      14,    1138,      54,
           182,    1404,     132,      58,     357, 3000440, 3000440,
       3000440, 3000

In [19]:
i=0
for seq in data_train[128:250]:
    for el in seq:
        if np.isnan(el):
            print(i)
                
#         if type(arr) is np.ndarray:
#             print(i)
    i+=1

In [5]:
import pickle
def save_pickle_file(file,file_path,protocol=None):
    with open(file_path,"wb") as f:
        if protocol:
            pickle.dump(file,f,protocol = protocol)
        else:
            pickle.dump(file,f)

def load_pickle_file(file_path, protocol=None):
     with open(file_path,"rb") as f:
        if protocol:
            return pickle.load(f)
        else:
            return pickle.load(f)
